# Static Model Test
This notebook tests the defined models using a gradio inteface where one can simply upload an image or video, choose a model and then get a classification of the perfromed exercise.

In [2]:
import gradio as gr
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

In [ ]:
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

MODEL_PATH = "..\skeleton_cnn_multiclass3.h5"
# bench_press: 0, bulgarian_squat: 1, lat_machine: 2, pull_up: 3, push_up: 4, split_squat: 5
CLASSES = ["pull_up", "push_up", "split_squat"]

KEYPOINT_DIM = 132  # 1 landmark with x,y,z,visibility

# ——— load trained model ———
model = tf.keras.models.load_model(MODEL_PATH)

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\hannes\AppData\Local\Temp\ipykernel_6204\2927921311.py:3: SyntaxWarning: invalid escape sequence '\s'
  MODEL_PATH = "..\skeleton_cnn_multiclass3.h5"


In [4]:
# ——— init Mediapipe Pose & etc. ———
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

pose = mp_pose.Pose(
    static_image_mode=True,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

In [5]:
# ——— helper to extract keypoints array from Mediapipe results ———
def extract_keypoints_from_results(results):
    kpts = []
    for lm in results.pose_landmarks.landmark:
        kpts += [lm.x, lm.y, lm.z, lm.visibility]
    arr = np.array(kpts, dtype=np.float32)
    return arr.reshape(1, KEYPOINT_DIM, 1)  # shape = (1,132,1)

In [14]:
def process_image(image):
    # Convert image to RGB (MediaPipe requires RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Get pose landmarks
    results = pose.process(image_rgb)
    
    if not results.pose_landmarks:
        return "No pose detected in the image", image
    
    # Extract keypoints
    keypoints = extract_keypoints_from_results(results)
    
    # Get model prediction
    predictions = model.predict(keypoints, verbose=0)[0]
    
    # Get top 3 predictions
    top_3_idx = np.argsort(predictions)[-3:][::-1]  # Get indices of top 3 predictions
    top_3_classes = [CLASSES[i] for i in top_3_idx]
    top_3_confidences = [float(predictions[i]) for i in top_3_idx]
    
    # Draw pose landmarks on image
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style()
    )
    
    # Add prediction text to image
    y_position = 30
    for i in range(3):
        text = f"{top_3_classes[i]}: {top_3_confidences[i]:.2%}"
        y_position += 40  # Move down for next line
    
    # Create detailed prediction text
    prediction_text = "Top 3 Predictions:\n"
    for i in range(3):
        prediction_text += f"{i+1}. {top_3_classes[i]}: {top_3_confidences[i]:.2%}\n"
    
    return prediction_text, annotated_image

In [15]:
# Create Gradio interface
demo = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="numpy"),
    outputs=[
        gr.Textbox(label="Prediction"),
        gr.Image(label="Annotated Image")
    ],
    title="Exercise Classification",
    description="Upload an image to classify the exercise being performed."
)

# Launch the interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
